In [1]:
from dotenv import load_dotenv
from crewai import Crew
from tasks import MemoryTasks
from agents import LTMAgents
load_dotenv()


CLEARING MEMORY


True

In [2]:
user_context = "I want to create a meal plan for myself for lunch but Im allergic to fish. Im also a vegetarian?"

In [3]:

tasks = MemoryTasks()
agents = LTMAgents()

# create agents
context_agent = agents.context_agent()
memory_agent = agents.memory_agent()
retrieval_agent = agents.retrieval_agent()

# create tasks
context_task = tasks.determine_memory_task(context_agent, user_context)
memory_task = tasks.store_memory(memory_agent)
retrieve_and_combine_relevant_memories = tasks.retrieve_and_combine_relevant_memories(retrieval_agent, user_context)

memory_task.context = [context_task]
    
crew = Crew(
    agents=[
        context_agent, 
        memory_agent,
        retrieval_agent
    ],
    tasks=[
        context_task,
        memory_task,
        retrieve_and_combine_relevant_memories
    ]
)

In [4]:
result = crew.kickoff()



> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: Allergy: Fish, Dietary Preference: Vegetarian.

> Finished chain.


> Entering new CrewAgentExecutor chain...
I need to focus on storing the memory provided by the context_specialist accurately in the database using the insert_memory tool. I should pay attention to the property type and property value to ensure the memory is stored correctly.

Action: 
insert_memory

Action Input: 
{
  "property_type": "Allergy",
  "property_value": "Fish"
}

INPUT-OUTPUT PAIR ADDED TO VECTOR DATABASE:
  ID
    1
  INPUT
    Allergy
  OUTPUT
    Fish

 

None

Final Answer: The memory "Allergy: Fish" has been successfully stored in the database.

> Finished chain.


> Entering new CrewAgentExecutor chain...
I need to retrieve memories related to the user's allergies and dietary preferences to create a personalized meal plan for lunch.

Action: retrieve_memories
Action Input: {"input_text": "allergies and dietary pr

In [6]:
print(result)

- Allergy: Fish
- Dietary Preference: Vegetarian


In [18]:
import asyncio
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model='gpt-3.5-turbo-0125')

def get_response(user_query, memories):

    template = """
    You are a helpful assistant. 
    Answer the following questions considering the user's question.
    You will also be provided a list of properties, preferences and restrictions correlating to the user that you should heavily consider
    when answering the user. These attributes are very important and should not be ignored

    User question: {user_question}
    properties, preferences and restrictions: {memories}
    """

    prompt = ChatPromptTemplate.from_template(template)

    llm = ChatOpenAI()
        
    chain = prompt | llm | StrOutputParser()
    
    return chain.invoke({
        "user_question": user_query,
        "memories": memories
    })


In [19]:
query='can you provide a meal plan for me?'
print(get_response(query, result))

Certainly! I can provide you with a vegetarian meal plan that excludes fish. Here is a sample meal plan for a day:

Breakfast:
- Greek yogurt with mixed berries and a sprinkle of granola
- Whole wheat toast with avocado slices

Lunch:
- Quinoa salad with roasted vegetables (bell peppers, zucchini, cherry tomatoes) and a lemon vinaigrette dressing
- Hummus and veggie wrap with spinach, cucumbers, and roasted red peppers

Dinner:
- Lentil and vegetable stir-fry with broccoli, bell peppers, carrots, and snap peas served over brown rice
- Roasted sweet potato wedges with a side of steamed asparagus

Snacks:
- Mixed nuts
- Fresh fruit (apple slices, grapes, or orange segments)

Remember to drink plenty of water throughout the day and adjust portion sizes according to your individual needs. Feel free to customize this meal plan to suit your tastes and preferences!
